# Logit and Nested Logit Tutorial

In [1]:
import pyblp
import numpy as np
import pandas as pd

pyblp.options.digits = 2
pyblp.options.verbose = False
pyblp.__version__

'0.6.0'

In this tutorial, we'll use data from :ref:`references:Nevo (2000)` to solve the paper's fake cereal problem. Locations of CSV files that contain the data are in the :mod:`data` module.

We will compare two simple models, the plain (IIA) logit model and the nested logit (GEV) model using the fake cereal dataset of :ref:`references:Nevo (2000)`.

## Theory of Plain Logit

Let's start with the plain logit model under independence of irrelevant alternatives (IIA). In this  model (indirect) utility is given by

$$U_{jti} = x_{jt} \beta - \alpha p_{jt} + \xi_{jt} + \epsilon_{jti},$$

where $\varepsilon_{jti}$ is distributed IID with the Type I extreme value (Gumbel) distribution. It is common to normalize the mean utility of the outside good to zero so that $U_{0ti} = \varepsilon_{0ti}$. This gives us aggregate marketshares

$$s_{jt} = \frac{\exp(x_{jt} \beta - \alpha p_{jt} + \xi_{jt})}{\sum_k \exp(x_{kt} \beta - \alpha p_{jt} + \xi_{kt})}.$$

If we take logs we get

$$\ln s_{jt} = x_{jt} \beta - \alpha p_{jt} + \xi_{jt} - \ln \sum_k \exp(x_{kt} \beta - \alpha p_{jt} + \xi_{kt})$$

and

$$\ln s_{0t} =  0 - \ln \sum_k \exp(x_{kt} \beta - \alpha p_{jt} + \xi_{kt}).$$

By differencing the above we get a linear estimating equation:

$$\ln s_{jt} - \ln s_{0t} = x_{jt}\beta  - \alpha p_{jt}  + \xi_{jt}.$$

Because the left hand side is data, we can estimate this model using linear IV GMM.

## Application of Plain Logit

A Logit :class:`Problem` can be created by simply excluding the formulation for the nonlinear parameters, $X_2$, along with any agent information. In other words, it requires only specifying the _linear component_ of demand.

We'll set up and solve a simple version of the fake data cereal problem from :ref:`references:Nevo (2000)`. Since we won't include any nonlinear characteristics or parameters, we don't have to worry about configuring an optimization routine.

There are some important reserved variable names:

- `market_ids` are the unique market identifiers which we subscript with $t$.
- `product_ids` are the unique product identifiers which we subscript with $j$. These are optional.
- `shares` specifies the marketshares which need to be between zero and one, and within a market ID, $\sum_{j} s_{jt} \leq 1$.
- `prices` are prices $p_{jt}$. These have some special properties and are _always_ treated as endogenous.
- `demand_instruments0`, `demand_instruments1`, and so on are numbered demand instruments. These represent only the _excluded_ instruments. The exogenous regressors in $X_1$ will be automatically added to the set of instruments.

We begin with two steps:

1. Load the `product data` which at a minimum consists of `market_ids`, `shares`, `prices`, and at least a single column of demand instruments, `demand_instruments0`.
2. Define a :class:`Formulation` for the $X_1$ (linear) demand model.

    - This and all other formulas are similar to R and [patsy](https://patsy.readthedocs.io/en/stable/) formulas.
    - It includes a constant by default. To exclude the constant either specify a `0` or a `-1`.
    - To efficiently include fixed effects, use the `absorb` option and specify which categorical variable(s) you would like to absorb.
    - Some model reduction may happen automatically. The constant will be excluded if you include fixed effects and some precautions are taken against collinearity. However, you will have to make sure that differently-named variables are not collinear.
    
3. Combine the :class:`Formulation` and `product data` to construct a :class:`Problem`.
4. Use :meth:`Problem.solve` to estimate paramters.

### Loading the Data

The `product_data` argument of :class:`Problem` should be a structured array-like object with fields that store data. Product data can be a structured [NumPy](https://www.numpy.org/) array, a [pandas](https://pandas.pydata.org/) DataFrame, or other similar objects.

In [2]:
product_data = pd.read_csv(pyblp.data.NEVO_PRODUCTS_LOCATION)
product_data.head()

,market_ids,city_ids,quarter,product_ids,firm_ids0,firm_ids1,brand_ids,shares,prices,sugar,...,demand_instruments10,demand_instruments11,demand_instruments12,demand_instruments13,demand_instruments14,demand_instruments15,demand_instruments16,demand_instruments17,demand_instruments18,demand_instruments19
0,C01Q1,1,1,F1B04,1,1,4,0.012417,0.072088,2,...,2.116358,-0.154708,-0.005796,0.014538,0.126244,0.067345,0.068423,0.034800,0.126346,0.035484
1,C01Q1,1,1,F1B06,1,1,6,0.007809,0.114178,18,...,-7.374091,-0.576412,0.012991,0.076143,0.029736,0.087867,0.110501,0.087784,0.049872,0.072579
2,C01Q1,1,1,F1B07,1,1,7,0.012995,0.132391,4,...,2.187872,-0.207346,0.003509,0.091781,0.163773,0.111881,0.108226,0.086439,0.122347,0.101842
3,C01Q1,1,1,F1B09,1,1,9,0.005770,0.130344,3,...,2.704576,0.040748,-0.003724,0.094732,0.135274,0.088090,0.101767,0.101777,0.110741,0.104332
4,C01Q1,1,1,F1B11,1,1,11,0.017934,0.154823,12,...,1.261242,0.034836,-0.000568,0.102451,0.130640,0.084818,0.101075,0.125169,0.133464,0.121111


The product data contains `market_ids`, `product_ids`, two sets of `firm_ids` (the second are IDs after a simple merger, which are not used in this notebook), `shares`, `prices`, a number of product characteristics, and some pre-computed excluded `demand_instruments0`, `demand_instruments1`, and so on. The `product_ids` will be used to construct fixed effects. 

For more information about the instruments and the example data as a whole, refer to the :mod:`data` module.

### Setting Up the Problem

We can combine the :class:`Formulation` and the `product_data` and construct a :class:`Problem`. We pass the :class:`Formulation` first and the `product_data` second. We can also display the properties of the problem by typing its name. 

In [3]:
logit_formulation = pyblp.Formulation('prices', absorb='C(product_ids)')
logit_formulation

prices + Absorb[C(product_ids)]

In [4]:
problem = pyblp.Problem(logit_formulation, product_data)
problem

Dimensions:
 N     T    K1    MD    ED 
----  ---  ----  ----  ----
2256  94    1     20    1  

Formulations:
       Column Indices:           0   
-----------------------------  ------
 X1: Linear Characteristics    prices

Two sets of properties are displayed:

1. Dimensions of the data.
2. Formulations of the problem.

The dimensions describe the shapes of matrices as layed out in :ref:`notation:Notation`. They include:

- $N$ the length of the dataset (all products and markets).
- $T$ the number of markets.
- $K_1$ the dimension of the linear demand parameters.
- $M_D$ is the dimension of the instrument variables (excluded instruments only, not exogenous regressors).
- $E_D$ is the number of fixed effects (1-D fixed effects, 2-D fixed effects, etc.).

There is only a single :class:`Formulation` for this model. 

- $X_1$ is the linear component of utility for demand and depends only on prices (after the fixed effects are removed).

### Solving the Problem

The :meth:`Problem.solve` method always returns a :class:`ProblemResults` class, which can be used to compute post-estimation outputs. See the [post-estimation](post_estimation.ipynb) for more information.

In [5]:
logit_results = problem.solve()
logit_results

Problem Results Summary:
                   Cumulative    Cumulative     Final  
Cumulative  GMM   Optimization  Optimization  Objective
   Time     Step   Iterations   Evaluations     Value  
----------  ----  ------------  ------------  ---------
 0:00:00     2         0             2        +4.2E+05 

Beta Estimates (Robust SEs in Parentheses):
  prices  
----------
 -3.0E+01 
(+1.0E+00)

### Multicollinearity Illustration

As an illustration, let's estimate a model with an obvious multicollinearity problem and see what happens. Suppose we include the variables `mushy` and `sugar` in the model. Because they don't vary within `product_ids`, they are absorbed into the fixed effects.

In [6]:
collinear_formulation = pyblp.Formulation('prices + mushy + sugar', absorb='C(product_ids)')
pyblp.Problem(collinear_formulation, product_data).solve()

Problem Results Summary:
                   Cumulative    Cumulative     Final  
Cumulative  GMM   Optimization  Optimization  Objective
   Time     Step   Iterations   Evaluations     Value  
----------  ----  ------------  ------------  ---------
 0:00:00     2         0             2        +4.2E+05 

Beta Estimates (Robust SEs in Parentheses):
  prices      mushy       sugar   
----------  ----------  ----------
 -3.0E+01    +0.0E+00    +0.0E+00 
(+1.0E+00)  (+0.0E+00)  (+0.0E+00)

Notice that we get the same results as before and we do not estimate coefficients on `mushy` or `sugar`. Although multicollinearity did not pose a problem here, in other cases it may create errors.

## Theory of Nested Logit

We can extend the logit model to allow for correlation within a group $h$ so that

$$U_{jti} = x_{jt} \beta - \alpha p_{jt} + \xi_{jt} +  \bar{\epsilon}_{h(j)ti} + (1 - \rho) \bar{\epsilon}_{jti}.$$

Now, we require that $\epsilon_{jti} = \bar{\epsilon}_{h(j)ti} +  (1 - \rho) \bar{\epsilon}_{jti}$ is distributed IID with the Type I extreme value (Gumbel) distribution. As $\rho \rightarrow 1$, all consumers stay within their group. As $\rho \rightarrow 0$, this collapses to the IIA logit. Note that if we wanted, we could allow $\rho$ to differ between groups with the notation $\rho_{h(j)}$.

This gives us aggregate marketshares as the product of two logits, the within group logit and the across group logit:

$$s_{jt} = \frac{\exp[V_{jt} / (1 - \rho)]}{\exp[V_{h(j)t} / (1 - \rho)]}\cdot\frac{\exp V_{h(j)t}}{1 + \sum_h \exp V_{ht}},$$

where $V_{jt} = x_{jt} \beta - \alpha p_{jt} + \xi_{jt}$.

After some work we again obtain the linear estimating equation:

$$\ln s_{jt} - \ln s_{0t} = x_{jt}\beta - \alpha p_{jt} +\rho \ln s_{j|h(j)t} + \xi_{jt},$$

where $s_{j|h(j)t} = s_{jt} / s_{h(j)t}$ and $s_{h(j)t}$ is the share of group $h$ in market $t$. See :ref:`references:Berry (1994)` or :ref:`references:Cardell (1997)` for more information.

Again, the left hand side is data, though the $\ln s_{j|h(j)t}$ is clearly endogenous which means we must instrument for it. Rather than include $\ln s_{j|h(j)t}$ along with the linear components of utility, $X_1$, whenever `nesting_ids` are included in `product_data`, $\rho$ is treated as a nonlinear $X_2$ parameter. This means that the linear component is given instead by

$$\ln s_{jt} - \ln s_{0t} -\rho \ln s_{j|h(j)t} = x_{jt}\beta - \alpha p_{jt}  + \xi_{jt}.$$

This is done for two reasons:

1. It forces the user to treat $\rho$ as an endogenous parameter.
2. It extends much more easily to the RCNL model of :ref:`references:Grigolon and Verboven (2014)`.

A common choice for an additional instrument is the number of products per nest.

## Application of Nested Logit

By including `nesting_ids` (another reserved name) as a field in `product_data`, we tell the package to estimate a nested logit model, and we don't need to change any of the formulas. We show how to construct the category groupings in two different ways:

1. We put all products in a single nest (only the outside good in the other nest).
2. We put products into two nests (either mushy or non-mushy).

We also construct an additional instrument based on the number of products per nest. Typically this is useful as a source of exogenous variation in the within group share $\ln s_{j|h(j)t}$. However, in this example because the number of products per nest do not vary across markets, if we include product fixed effects, this instrument is irrelevant.

We'll define a function that constructs the additional instrument and solves the nested logit problem. We'll exclude product ID fixed effects, which are collinear with `mushy,` and we'll choose $\rho = 0.7$ as the initial value at which the optimization routine will start.

In [7]:
def solve_nl(df):
    groups = df.groupby(['market_ids', 'nesting_ids'])
    df['demand_instruments20'] = groups['shares'].transform(np.size)
    nl_formulation = pyblp.Formulation('0 + prices')
    problem = pyblp.Problem(nl_formulation, df)
    return problem.solve(rho=0.7)

First, we'll solve the problem when there's a single nest for all products, with the outside good in its own nest.

In [8]:
df1 = product_data.copy()
df1['nesting_ids'] = 1
nl_results1 = solve_nl(df1)
nl_results1

Problem Results Summary:
                   Cumulative    Cumulative     Final    Gradient
Cumulative  GMM   Optimization  Optimization  Objective  Infinity
   Time     Step   Iterations   Evaluations     Value      Norm  
----------  ----  ------------  ------------  ---------  --------
 0:00:01     2         5             9        +4.6E+05   +1.0E-05

Rho Estimates (Robust SEs in Parentheses):
All Groups
----------
 +9.8E-01 
(+1.4E-02)

Beta Estimates (Robust SEs in Parentheses):
  prices  
----------
 -1.2E+00 
(+4.0E-01)

When we inspect the :class:`Problem`, the only change from the plain logit model is the inclusion of $H$, the number of nesting categories.

In [9]:
nl_results1.problem

Dimensions:
 N     T    K1    MD    H 
----  ---  ----  ----  ---
2256  94    1     21    1 

Formulations:
       Column Indices:           0   
-----------------------------  ------
 X1: Linear Characteristics    prices

Next, we'll solve the problem when there are two nests for mushy and non-mushy.

In [10]:
df2 = product_data.copy()
df2['nesting_ids'] = df2['mushy']
nl_results2 = solve_nl(df2)
nl_results2

Problem Results Summary:
                   Cumulative    Cumulative     Final    Gradient
Cumulative  GMM   Optimization  Optimization  Objective  Infinity
   Time     Step   Iterations   Evaluations     Value      Norm  
----------  ----  ------------  ------------  ---------  --------
 0:00:01     2         6             9        +1.6E+06   +5.7E-08

Rho Estimates (Robust SEs in Parentheses):
All Groups
----------
 +8.9E-01 
(+1.9E-02)

Beta Estimates (Robust SEs in Parentheses):
  prices  
----------
 -7.8E+00 
(+4.8E-01)

For both cases we find that $\hat{\rho} > 0.8$.

Finally, we'll also look at the adjusted parameter on prices, $\alpha / (1-\rho)$.

In [11]:
nl_results1.beta[0] / (1 - nl_results1.rho)

array([[-67.39338888]])

In [12]:
nl_results2.beta[0] / (1 - nl_results2.rho)

array([[-72.27074638]])

### Treating Within Group Shares as Exogenous

The package is designed to prevent the user from treating the within group share, $\log s_{j|h(j)t}$, as an exogenous variable. For example, if we were to compute a `group_share` variable and use the algebraic functionality of :class:`Formulation` by including the expression `log(shares / group_share)` in our formula for $X_1$, the package would raise an error because `shares` should not be included in this formulation.

In order to demonstrate why this is a bad idea, we override this feature by calculating $\log s_{j|h(j)t}$ and including it as an additional variable in $X_1$. To do so, we'll first re-define our function for setting up and solving the nested logit problem.

In [13]:
def solve_nl2(df):
    groups = df.groupby(['market_ids', 'nesting_ids'])
    df['group_share'] = groups['shares'].transform(np.sum)
    df['within_share'] = df['shares'] / df['group_share']
    df['demand_instruments20'] = groups['shares'].transform(np.size)
    nl2_formulation = pyblp.Formulation('0 + prices + log(within_share)')
    problem = pyblp.Problem(nl2_formulation, df.drop(columns=['nesting_ids']))
    return problem.solve()

Again, we'll solve the problem when there's a single nest for all products, with the outside good in its own nest.

In [14]:
nl2_results1 = solve_nl2(df1)
nl2_results1

Problem Results Summary:
                   Cumulative    Cumulative     Final  
Cumulative  GMM   Optimization  Optimization  Objective
   Time     Step   Iterations   Evaluations     Value  
----------  ----  ------------  ------------  ---------
 0:00:00     2         0             2        +4.6E+05 

Beta Estimates (Robust SEs in Parentheses):
  prices    log(within_share)
----------  -----------------
 -1.0E+00       +9.9E-01     
(+2.4E-01)     (+7.9E-03)    

And again, we'll solve the problem when there are two nests for mushy and non-mushy.

In [15]:
nl2_results2 = solve_nl2(df2)
nl2_results2

Problem Results Summary:
                   Cumulative    Cumulative     Final  
Cumulative  GMM   Optimization  Optimization  Objective
   Time     Step   Iterations   Evaluations     Value  
----------  ----  ------------  ------------  ---------
 0:00:00     2         0             2        +1.6E+06 

Beta Estimates (Robust SEs in Parentheses):
  prices    log(within_share)
----------  -----------------
 -6.8E+00       +9.3E-01     
(+2.9E-01)     (+1.1E-02)    

One can observe that we obtain parameter estimates which are quite different than above.

In [16]:
nl2_results1.beta[0] / (1 - nl2_results1.beta[1])

array([-86.37368446])

In [17]:
nl2_results2.beta[0] / (1 - nl2_results2.beta[1])

array([-100.14496892])